In [141]:
import pandas as pd
from sqlalchemy import create_engine

In [142]:
consumption_file = "Resources/alcohol_consumption.csv"
consumption_data_df = pd.read_csv(consumption_file)
consumption_data_df.head()

,Year,Locationabbr,Locationdesc,Class,Topic,Question,Response,Break_Out,Break_Out_Category,Sample_Size,...,Data_Value_Footnote,DataSource,ClassId,TopicId,LocationID,BreakoutID,BreakOutCategoryID,QuestionID,ResponseID,GeoLocation
0,2018,AK,Alaska,Alcohol Consumption,Heavy Drinking,Heavy drinkers (adult men having more than 14 ...,Meet criteria for heavy drinking,"American Indian or Alaskan Native, non-Hispanic",Race/Ethnicity,29,...,NaN,BRFSS,CLASS01,TOPIC30,2,RACE03,CAT4,_RFDRHV6,RESP206,"(64.84507995700051, -147.72205903599973)"
1,2018,AK,Alaska,Alcohol Consumption,Alcohol Consumption,Adults who have had at least one drink of alco...,Yes,35-44,Age Group,219,...,NaN,BRFSS,CLASS01,TOPIC03,2,AGE03,CAT3,DRNKANY5,RESP046,"(64.84507995700051, -147.72205903599973)"
2,2018,AK,Alaska,Alcohol Consumption,Binge Drinking,Binge drinkers (males having five or more drin...,Yes,18-24,Age Group,35,...,NaN,BRFSS,CLASS01,TOPIC07,2,AGE01,CAT3,_RFBING5,RESP046,"(64.84507995700051, -147.72205903599973)"
3,2018,AK,Alaska,Alcohol Consumption,Heavy Drinking,Heavy drinkers (adult men having more than 14 ...,Meet criteria for heavy drinking,Some post-H.S.,Education Attained,49,...,NaN,BRFSS,CLASS01,TOPIC30,2,EDUCA3,CAT5,_RFDRHV6,RESP206,"(64.84507995700051, -147.72205903599973)"
4,2018,AK,Alaska,Alcohol Consumption,Heavy Drinking,Heavy drinkers (adult men having more than 14 ...,Meet criteria for heavy drinking,"$15,000-$24,999",Household Income,15,...,Prevalence estimate not available if the unwei...,BRFSS,CLASS01,TOPIC30,2,INCOME2,CAT6,_RFDRHV6,RESP206,"(64.84507995700051, -147.72205903599973)"


In [143]:
excessive_file = "Resources/excessive_drinking.csv"
excessive_data_df = pd.read_csv(excessive_file)
excessive_data_df.head()

,Edition,Report Type,Measure Name,Age Range 18-44,Age Range 45-64,State Name,Rank,Value,Source Year
0,2019,2019 Annual,Excessive Drinking,Ages 18-44,Ages 45-64,Alabama,3,13.8,2018
1,2019,2019 Annual,Excessive Drinking,Ages 18-44,Ages 45-64,Alaska,24,17.7,2018
2,2019,2019 Annual,Excessive Drinking,Ages 18-44,Ages 45-64,Arizona,21,17.2,2018
3,2019,2019 Annual,Excessive Drinking,Ages 18-44,Ages 45-64,Arkansas,10,15.8,2018
4,2019,2019 Annual,Excessive Drinking,Ages 18-44,Ages 45-64,California,23,17.6,2018


In [144]:
consumption_cols = ["Year", "Locationdesc", "Topic"]
consumption_transformed = consumption_data_df[consumption_cols].copy()

consumption_transformed = consumption_transformed.rename(columns = {"Locationdesc": "state", "Topic": "consumption_amount", 
                                                                    "Year": "year"})
consumption_transformed.set_index("state", inplace=True)

consumption_transformed.head()

,year,consumption_amount
state,,
Alaska,2018,Heavy Drinking
Alaska,2018,Alcohol Consumption
Alaska,2018,Binge Drinking
Alaska,2018,Heavy Drinking
Alaska,2018,Heavy Drinking


In [145]:
excessive_cols = ["Source Year", "Measure Name", "State Name", "Rank"]
excessive_transformed = excessive_data_df[excessive_cols].copy()

excessive_transformed = excessive_transformed.rename(columns = {"Source Year": "year", "Measure Name": "consumption_type", 
                                                                    "State Name": "state", "Rank": "rank"})
excessive_transformed.set_index("state", inplace=True)

excessive_transformed.head()

,year,consumption_type,rank
state,,,
Alabama,2018,Excessive Drinking,3
Alaska,2018,Excessive Drinking,24
Arizona,2018,Excessive Drinking,21
Arkansas,2018,Excessive Drinking,10
California,2018,Excessive Drinking,23


In [146]:
connection_string = "postgres:Dog1wood@localhost:5432/consumption_db"
engine = create_engine(f'postgresql://{connection_string}')

In [147]:
engine.table_names()

['excessive_drinking', 'consumption']

In [148]:
consumption_transformed.to_sql(name='consumption', con=engine, if_exists='append', index=True)

In [149]:
excessive_transformed.to_sql(name='excessive_drinking', con=engine, if_exists='append', index=True)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "excessive_drinking_pkey"
DETAIL:  Key (year)=(2018) already exists.

[SQL: INSERT INTO excessive_drinking (state, year, consumption_type, rank) VALUES (%(state)s, %(year)s, %(consumption_type)s, %(rank)s)]
[parameters: ({'state': 'Alabama', 'year': 2018, 'consumption_type': 'Excessive Drinking', 'rank': 3}, {'state': 'Alaska', 'year': 2018, 'consumption_type': 'Excessive Drinking', 'rank': 24}, {'state': 'Arizona', 'year': 2018, 'consumption_type': 'Excessive Drinking', 'rank': 21}, {'state': 'Arkansas', 'year': 2018, 'consumption_type': 'Excessive Drinking', 'rank': 10}, {'state': 'California', 'year': 2018, 'consumption_type': 'Excessive Drinking', 'rank': 23}, {'state': 'Colorado', 'year': 2018, 'consumption_type': 'Excessive Drinking', 'rank': 42}, {'state': 'Connecticut', 'year': 2018, 'consumption_type': 'Excessive Drinking', 'rank': 35}, {'state': 'Delaware', 'year': 2018, 'consumpt